## Test accuracy on 20 CIFAR-100 classes using EfficieentNetB0

### Import all the necessary libraries

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.datasets import cifar100
# import torch

# print("CUDA available:", torch.cuda.is_available())
# print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPU Devices:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.19.0
Num GPUs Available: 1
GPU Devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


###  Load CIFAR-100 fine labels (100-class)

In [4]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

### Choose 20 classes

In [5]:
selected_classes = list(range(20))
train_mask = np.isin(y_train, selected_classes).flatten()
test_mask = np.isin(y_test, selected_classes).flatten()

x_train_20 = x_train[train_mask]
y_train_20 = y_train[train_mask]
x_test_20 = x_test[test_mask]
y_test_20 = y_test[test_mask]

### Map label to 0 - 19

In [6]:
label_map = {cls: i for i, cls in enumerate(selected_classes)}
y_train_20 = np.array([label_map[y[0]] for y in y_train_20])
y_test_20 = np.array([label_map[y[0]] for y in y_test_20])

### Resize to 128x128

In [7]:
x_train_20 = tf.image.resize(x_train_20, (128, 128)).numpy()
x_test_20 = tf.image.resize(x_test_20, (128, 128)).numpy()

I0000 00:00:1748279633.947654   28707 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7165 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1
2025-05-26 23:13:54.130965: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1966080000 exceeds 10% of free system memory.
2025-05-26 23:13:57.655678: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 393216000 exceeds 10% of free system memory.


### Preprocess

In [8]:
x_train_20 = preprocess_input(x_train_20)
x_test_20 = preprocess_input(x_test_20)

### Converts Label

In [9]:
y_train_20_cat = tf.keras.utils.to_categorical(y_train_20, 20)
y_test_20_cat = tf.keras.utils.to_categorical(y_test_20, 20)

### Load EfficieentNetB0 base model

In [ ]:
base_model = EfficientNetB1(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # freeze base

# 8. Build model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(20, activation='softmax')
])

# 9. Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 10. Train
model.fit(x_train_20, y_train_20_cat, epochs=5, batch_size=64, validation_split=0.1)

# 11. Evaluate
loss, acc = model.evaluate(x_test_20, y_test_20_cat)
print("Test accuracy on 20 CIFAR-100 classes using EfficientNetB1:", acc)

2025-05-26 23:14:01.591537: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1769472000 exceeds 10% of free system memory.
2025-05-26 23:14:03.044446: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1769472000 exceeds 10% of free system memory.


Epoch 1/5


I0000 00:00:1748279654.069144   28783 service.cc:152] XLA service 0x790194097440 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748279654.069164   28783 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, Compute Capability 6.1
2025-05-26 23:14:14.539210: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748279656.947508   28783 cuda_dnn.cc:529] Loaded cuDNN version 90300


  3/141 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.0868 - loss: 3.1440    

I0000 00:00:1748279670.031360   28783 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 0.6004 - loss: 1.4556

2025-05-26 23:14:50.125684: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 196608000 exceeds 10% of free system memory.


141/141 ━━━━━━━━━━━━━━━━━━━━ 58s 228ms/step - accuracy: 0.6013 - loss: 1.4521 - val_accuracy: 0.8360 - val_loss: 0.5182
Epoch 2/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.8515 - loss: 0.4838 - val_accuracy: 0.8460 - val_loss: 0.4842
Epoch 3/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.8806 - loss: 0.3759 - val_accuracy: 0.8520 - val_loss: 0.4556
Epoch 4/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.9067 - loss: 0.2938 - val_accuracy: 0.8510 - val_loss: 0.4663
Epoch 5/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.9243 - loss: 0.2322 - val_accuracy: 0.8460 - val_loss: 0.4685
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 122ms/step - accuracy: 0.8601 - loss: 0.4292
Test accuracy on 20 CIFAR-100 classes using EfficientNetB1: 0.8610000014305115


: 